# Predicting Exoplanet Candidates with Machine Learning

In this project, we aim to build a Machine Learning model to predict whether a candidate is a confirmed exoplanet or not. We will use publicly available data to create a classification model that can automate the process of identifying confirmed planets. This document will serve as a guide through the entire workflow, starting from data exploration and preparation to model building and evaluation.

The data used in this project can be found at the following link:

[Dataset Link](https://drive.google.com/file/d/1Ui9lIx8LeKaV6UuXT5VNbyz6XplGO2N5/view)

The dataset contains public information about exoplanets provided by the California Institute of Technology (CalTech) and the National Aeronautics and Space Administration (NASA) of the United States.

If you are interested in learning more about the variables included in the dataset, you can refer to the following link for a detailed description:

[Variable Description](https://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html)



# Identification and Data Preparation

In this section, we will focus on the identification and preparation of data for building a Machine Learning model to detect confirmed exoplanets. We will perform an exploratory analysis of the variables and prepare the data to create a clean dataset ready for modeling.

In [24]:
import pandas as pd

In [25]:
# Load the dataset
df = pd.read_csv("exoplanetas.csv", delimiter=';')
# Display the first few rows to understand the structure of the dataset
df.head()

,koi_disposition,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,...,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,deg_sc,koi_kepmag
0,CONFIRMED,"54,4183827","0,0002479","-0,0002479","162,51384","0,00352","-0,00352","0,586","0,059","-0,443",...,"4,467","0,064","-0,096","0,927","0,105","-0,061","291,93423","48,141651",NaN,"15,347"
1,FALSE POSITIVE,"19,89913995","1,49E-05","-1,49E-05","175,850252","0,000581","-0,000581","0,969","5,126","-0,077",...,"4,544","0,044","-0,176","0,868","0,233","-0,078","297,00482","48,134129",NaN,"15,436"
2,FALSE POSITIVE,"1,736952453","2,63E-07","-2,63E-07","170,307565","0,000115","-0,000115","1,276","0,115","-0,092",...,"4,564","0,053","-0,168","0,791","0,201","-0,067","285,53461","48,28521",NaN,"15,597"
3,CONFIRMED,"2,525591777","3,76E-06","-3,76E-06","171,59555","0,00113","-0,00113","0,701","0,235","-0,478",...,"4,438","0,07","-0,21","1,046","0,334","-0,133","288,75488","48,2262",NaN,"15,509"
4,CONFIRMED,"4,13443512","1,05E-05","-1,05E-05","172,97937","0,0019","-0,0019","0,762","0,139","-0,532",...,"4,486","0,054","-0,229","0,972","0,315","-0,105","296,28613","48,22467",NaN,"15,714"


In [26]:
# Display the column names to identify the available variables
print(df.columns), df.shape[1]

Index(['koi_disposition', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec', 'deg_sc',
       'koi_kepmag'],
      dtype='object')


(None, 38)

The dataset contains various features related to exoplanet candidates, such as their orbital period, radius, temperature, and other stellar properties.

### Description of Columns

- **koi_disposition**: The final classification of the candidate, which can be CONFIRMED, FALSE POSITIVE, or CANDIDATE.
- **koi_period**: Orbital period of the candidate (in days).
- **koi_period_err1 / koi_period_err2**: Upper and lower uncertainty in the orbital period.
- **koi_time0bk**: Time corresponding to the center of the first detected transit.
- **koi_time0bk_err1 / koi_time0bk_err2**: Upper and lower uncertainty in the transit epoch.
- **koi_impact**: Impact parameter, representing the sky-projected distance between the center of the stellar disc and the center of the planet disc at conjunction.
- **koi_impact_err1 / koi_impact_err2**: Upper and lower uncertainty in the impact parameter.
- **koi_duration**: Duration of the observed transits (in hours).
- **koi_duration_err1 / koi_duration_err2**: Upper and lower uncertainty in the transit duration.
- **koi_depth**: Transit depth, representing the fraction of stellar flux lost at the minimum of the planetary transit.
- **koi_depth_err1 / koi_depth_err2**: Upper and lower uncertainty in the transit depth.
- **koi_prad**: Planetary radius (in Earth radii).
- **koi_prad_err1 / koi_prad_err2**: Upper and lower uncertainty in the planetary radius.
- **koi_teq**: Estimated equilibrium temperature of the planet (in Kelvin).
- **koi_insol**: Insolation flux received by the planet, relative to Earth.
- **koi_insol_err1 / koi_insol_err2**: Upper and lower uncertainty in the insolation flux.
- **koi_model_snr**: Signal-to-noise ratio of the transit.
- **koi_tce_plnt_num**: Number of planets identified in the system.
- **koi_steff**: Stellar effective temperature (in Kelvin).
- **koi_steff_err1 / koi_steff_err2**: Upper and lower uncertainty in the stellar effective temperature.
- **koi_slogg**: Stellar surface gravity (logarithm, in cm/s²).
- **koi_slogg_err1 / koi_slogg_err2**: Upper and lower uncertainty in the stellar surface gravity.
- **koi_srad**: Stellar radius (in solar radii).
- **koi_srad_err1 / koi_srad_err2**: Upper and lower uncertainty in the stellar radius.
- **ra**: Right ascension of the target star (in degrees).
- **dec**: Declination of the target star (in degrees).
- **deg_sc**: Sky coordinate.
- **koi_kepmag**: Kepler magnitude of the target star.

## 1) Identifying the Target Variable

The target variable for our model is `koi_disposition`, which indicates whether an exoplanet candidate is CONFIRMED, FALSE POSITIVE, or still a CANDIDATE. For our purposes, we will focus on classifying the candidates as either CONFIRMED, FALSE POSITIVE, or CANDIDATE to fully capture the disposition of each exoplanet candidate.